# INDEX THE DOCUMENTS

In [1]:
!pip install elasticsearch

## Import Libraries

In [2]:
import pandas as pd
from elasticsearch import Elasticsearch
import tensorflow
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
tqdm.pandas()

C:\Users\Honor\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Honor\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Honor\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Connect to elasticsearch

In [3]:
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "ysP1ylGF4F9UCzbD3RXLzCVW"

# Found in the 'Manage Deployment' page
CLOUD_ID="8fc609e4e1a947fab7f3ab52e7e1c3d7:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGFmZDgyZGYyNzFlNjRiYjRhNTcwNjg2Yzk4ZWFmOTI0JDljYmQ0NDhhMWM3MjRjNjE5YTk4NTAyOWE3NmFkM2E0"

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Successful response!
client.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'afd82df271e64bb4a570686c98eaf924', 'cluster_uuid': 'xAhZB5KlRYeDghOsIdDdRg', 'version': {'number': '8.11.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '64cf052f3b56b1fd4449f5454cb88aca7e739d9a', 'build_date': '2023-12-08T11:33:53.634979452Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
client.ping()

True

# EDA

# Prepare the data

In [5]:
df = pd.read_json("../data/HarvardHealthBlog/medBlog.json")

In [6]:
df.shape

(13, 11)

In [7]:
df.head(2)

,_id,title,image,content,createdAt,updatedAt,author,tags,comments,likes,_class
0,{'$oid': '658b94f76c78f1229b2d6ddf'},Exploring the Impact of Nutrition on Mental He...,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,The connection between nutrition and mental he...,{'$date': '2023-12-27T03:07:35.873Z'},{'$date': '2023-12-27T03:07:35.873Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Nutrition, Diet and Mood, Mental Health, Brai...",[],0,com.ensa.medicalblog.entity.PostEntity
1,{'$oid': '658b955e6c78f1229b2d6de8'},The Microbiome: Unveiling the World Within Us,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,Embark on a fascinating journey into the intri...,{'$date': '2023-12-27T03:09:18.668Z'},{'$date': '2023-12-27T03:09:18.668Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Antibiotics, Gut Microbiota, Human Health, Mi...",[],0,com.ensa.medicalblog.entity.PostEntity


In [22]:
df = df.rename(columns={"_id": "postID"})

# SKIP ME _______

### Keep the essential columns

In [8]:
df.columns

Index(['_id', 'title', 'image', 'content', 'createdAt', 'updatedAt', 'author',
       'tags', 'comments', 'likes', '_class'],
      dtype='object')

In [11]:
df = df[['Title1', 'Image1', 'inlineblock_URL', 'fontbold1', 'block1', 'Field1']]

In [12]:
df.head(2)

,Title1,Image1,inlineblock_URL,fontbold1,block1,Field1
0,Immune boosts or busts? From IV drips and deto...,https://domf5oio6qrcr.cloudfront.net/medialibr...,https://www.health.harvard.edu/blog/immune-boo...,Staying Healthy,"\n Published September 11, 2023\n",Ads for products that promise to supercharge t...
1,"After prostate cancer treatment, a new standar...",https://domf5oio6qrcr.cloudfront.net/medialibr...,https://www.health.harvard.edu/blog/after-pros...,Men's Health,"\n Published December 20, 2023\n",When prostate cancer recurs after initial trea...


### Rename columns

In [13]:
df.columns

Index(['Title1', 'Image1', 'inlineblock_URL', 'fontbold1', 'block1', 'Field1'], dtype='object')

In [14]:
df = df.rename(columns={"Title1": "title",
                   "Image1": "imageURL",
                   "inlineblock_URL": "articleURL",
                   "fontbold1": "category",
                   "block1": "publishedDate",
                   "Field1": "content"
                  })

In [15]:
df.head(2)

,title,imageURL,articleURL,category,publishedDate,content
0,Immune boosts or busts? From IV drips and deto...,https://domf5oio6qrcr.cloudfront.net/medialibr...,https://www.health.harvard.edu/blog/immune-boo...,Staying Healthy,"\n Published September 11, 2023\n",Ads for products that promise to supercharge t...
1,"After prostate cancer treatment, a new standar...",https://domf5oio6qrcr.cloudfront.net/medialibr...,https://www.health.harvard.edu/blog/after-pros...,Men's Health,"\n Published December 20, 2023\n",When prostate cancer recurs after initial trea...


### Pre-process

In [16]:
df.publishedDate[0]

'\n            Published September 11, 2023\n    '

- remove the newline character ("\n"), spaces, and the word "Published|Updated" from the "df['publishedDate']" column

In [17]:
df['publishedDate'] = df['publishedDate'].str.replace(r'\n|\s|Published|Updated', '', regex=True)

In [18]:
df['publishedDate']

0      September11,2023
1       December20,2023
2       December19,2023
3       December18,2023
4         January5,2022
             ...       
182       August17,2022
183       August16,2022
184       August15,2022
185       August11,2022
186       August23,2022
Name: publishedDate, Length: 187, dtype: object

### Check NAN Values

In [9]:
df.isna().sum().sum()

0

In [10]:
df.head()

,_id,title,image,content,createdAt,updatedAt,author,tags,comments,likes,_class
0,{'$oid': '658b94f76c78f1229b2d6ddf'},Exploring the Impact of Nutrition on Mental He...,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,The connection between nutrition and mental he...,{'$date': '2023-12-27T03:07:35.873Z'},{'$date': '2023-12-27T03:07:35.873Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Nutrition, Diet and Mood, Mental Health, Brai...",[],0,com.ensa.medicalblog.entity.PostEntity
1,{'$oid': '658b955e6c78f1229b2d6de8'},The Microbiome: Unveiling the World Within Us,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,Embark on a fascinating journey into the intri...,{'$date': '2023-12-27T03:09:18.668Z'},{'$date': '2023-12-27T03:09:18.668Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Antibiotics, Gut Microbiota, Human Health, Mi...",[],0,com.ensa.medicalblog.entity.PostEntity
2,{'$oid': '658b95b86c78f1229b2d6df1'},The Power of Mindfulness in Stress Reduction,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,Embark on a journey of self-discovery as we ex...,{'$date': '2023-12-27T03:10:48.267Z'},{'$date': '2023-12-27T03:10:48.267Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Stress Reduction, Mindful Breathing, Emotiona...",[],0,com.ensa.medicalblog.entity.PostEntity
3,{'$oid': '658b96156c78f1229b2d6dfa'},Unlocking the Potential of Plant-Based Nutrition,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,Embark on a journey into the world of plant-ba...,{'$date': '2023-12-27T03:12:21.333Z'},{'$date': '2023-12-27T03:12:21.333Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Plant-Based Nutrition, Vegetarian, Health Ben...",[],0,com.ensa.medicalblog.entity.PostEntity
4,{'$oid': '658b96926c78f1229b2d6e03'},The Impact of Technology on Mental Health: Nav...,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,"In the ever-evolving digital landscape, techno...",{'$date': '2023-12-27T03:14:26.272Z'},{'$date': '2023-12-27T03:14:26.272Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Digital Well-being, Technology, Online Commun...",[],0,com.ensa.medicalblog.entity.PostEntity


# START HERE _______

# Convert text to Vector using S-BERT Model

In [11]:
model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
df["description_vector"] = df["content"].progress_apply(lambda x: model.encode(x))

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  2.14it/s]


In [23]:
df.head(2)

,postID,title,image,content,createdAt,updatedAt,author,tags,comments,likes,_class,description_vector
0,{'$oid': '658b94f76c78f1229b2d6ddf'},Exploring the Impact of Nutrition on Mental He...,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,The connection between nutrition and mental he...,{'$date': '2023-12-27T03:07:35.873Z'},{'$date': '2023-12-27T03:07:35.873Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Nutrition, Diet and Mood, Mental Health, Brai...",[],0,com.ensa.medicalblog.entity.PostEntity,"[-0.009982353, 0.064412005, -0.020627184, -0.0..."
1,{'$oid': '658b955e6c78f1229b2d6de8'},The Microbiome: Unveiling the World Within Us,http://res.cloudinary.com/ddhyzyvmp/image/uplo...,Embark on a fascinating journey into the intri...,{'$date': '2023-12-27T03:09:18.668Z'},{'$date': '2023-12-27T03:09:18.668Z'},"{'$ref': 'user', '$id': {'$oid': '658b3d003309...","[Antibiotics, Gut Microbiota, Human Health, Mi...",[],0,com.ensa.medicalblog.entity.PostEntity,"[0.06837606, 0.030840887, -0.008685436, -0.045..."


In [38]:
client.ping()

True

# Create new index

In [25]:
from indexMapping import indexMapping

In [26]:
# es.indices.create(index="all_documents", mappings= indexMapping)

In [27]:
index_name = "med-blogs"
if not client.indices.exists(index=index_name):
    client.indices.create(index=index_name, mappings=indexMapping)
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

Index 'med-blogs' already exists.


In [28]:
records_list = df.to_dict("records")

In [32]:
len(records_list)

13

In [33]:
for record in records_list:
    try:
        client.index(index= index_name, document=record)
    except Exception as e:
        print(e)

Connection timed out


In [34]:
client.count(index= index_name)

ObjectApiResponse({'count': 26, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

# Searching documents:

In [36]:
df.columns

Index(['postID', 'title', 'image', 'content', 'createdAt', 'updatedAt',
       'author', 'tags', 'comments', 'likes', '_class', 'description_vector'],
      dtype='object')

In [37]:
user_input = "toys"
vector_input = model.encode(user_input)

query = {
    "field": "description_vector",
    "query_vector": vector_input,
    "k":5,
    "num_candidates": 200,
}

res = client.knn_search(index= index_name , knn=query, source=['title', 'imageURL', 'articleURL', 'category', 'publishedDate',
       'content'])

results = res["hits"]["hits"]

for result in results:
    if "_source" in result:
        try:
            print(f"Document score: {result['_score']}")
            print(f"Document Title: {result['_source']['title']}")
            print(f"Document Text: {result['_source']['content']}")
            print(50*"_")
        except Exception as e:
            print(e)
    

# results
# relevants_docs
# score = res["hits"][:max_score]
# score

Document score: 0.36365005
Document Title: Digital Detox: Navigating the Benefits of Unplugging
Document Text: Embark on a journey of digital detoxification and explore the profound benefits of unplugging from the digital world. This post delves into the impact of constant connectivity on mental health, sleep patterns, and overall well-being. Discover practical tips for a successful digital detox, fostering a healthier relationship with technology and embracing moments of mindful presence.
__________________________________________________
Document score: 0.36365005
Document Title: Digital Detox: Navigating the Benefits of Unplugging
Document Text: Embark on a journey of digital detoxification and explore the profound benefits of unplugging from the digital world. This post delves into the impact of constant connectivity on mental health, sleep patterns, and overall well-being. Discover practical tips for a successful digital detox, fostering a healthier relationship with technology an

C:\Users\Honor\AppData\Local\Temp\ipykernel_21604\184663085.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = client.knn_search(index= index_name , knn=query, source=['title', 'imageURL', 'articleURL', 'category', 'publishedDate',
